In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
data_train = np.load('../data/pixelized_sample.npz')
data_labels = np.load('../data/pixelized_sample_class.npz')

In [ ]:
x_train = data_train['arr_0'][:4000]
x_labels = data_labels['arr_0'][:4000]


In [ ]:
category_mapping = {90: 'SN1a', 67: 'SN1a-91bg', 52: 'SN1ax', 42: 'SN2', 62: 'SN1bc', 95: 'SLSN1', 15: 'TDE', 64: 'KN', 88: 'AGN', 92: 'RRL', 65: 'M-dwarf', 16: 'EB', 53: 'Mira', 6: 'Microlens'}

In [ ]:
y_train = []
for i,obj_class in enumerate(x_labels):
    #class_ = meta_[meta_['object_id'] == obj_id]['target'].values[0]
    
    if category_mapping[obj_class] == 'AGN':
        y_train.append(1)
    else:
        y_train.append(0)
    
y_train = np.array(y_train)

In [ ]:
# Model / data parameters
num_classes = 2
input_shape = (256, 1300, 6)

y_train = keras.utils.to_categorical(y_train, num_classes)

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(30, 30), activation="relu"),
        layers.MaxPooling2D(pool_size=(8, 8)),
        layers.Conv2D(64, kernel_size=(15, 15), activation="relu"),
        layers.MaxPooling2D(pool_size=(4, 4)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

In [ ]:
batch_size = 32
epochs = 3

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
'''
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)
'''